In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [3]:
df=pd.read_csv('ecommerce-data-with-anomalies.csv')
df.head()

,ID,Name,Age,Gender,City,Country,CreditCard,FullAddress,UserSignUpDate,UserLastPurchaseDate,...,UserLastLoginDate,UserPurchaseFrequency,Product Name,Category,Subcategory,Price,Date_Purchase,Quantity,Rating,Comment
0,0,Elizabeth Harris,42,F,Chungberg,Mismatched Country,2227218447017949,"75874 Chad Mountain Suite 588, West Robertches...",2021-10-31,2022-09-10,...,2022-10-23,Yearly,NaN,Furniture,Chair,-45,2022-11-16,-1,5,Decade cold it town people.
1,1,Michele Best,72,M,Port Sergio,Egypt,2556439104848944,"4039 David Fords Apt. 678, Lake Christopherber...",2023-03-25,2023-05-06,...,2023-05-15,Yearly,FreshVegetables,Grocery,Vegetables,-79,2023-03-26,1,1,Really development floor of.
2,2,Jake Taylor,-5,M,Evansmouth,Czech Republic,2273966520760545,"748 Mendez Ridges Apt. 994, Lake Jonathanton, ...",2021-09-14,2023-04-21,...,2023-07-20,Yearly,TastyBeverages,Grocery,Beverages,-1,2023-07-12,10,4,Cut television step increase structure serve u...
3,3,Christian Curry MD,64,M,Jensenfurt,Brunei Darussalam,2238348117052863,"735 Jones Mews, New Ginafurt, AR 99785",2022-03-10,2023-01-24,...,2023-04-25,Yearly,ComfortChair,Furniture,Chair,770,2023-06-26,0,0,Popular outside industry listen.
4,4,Craig Jones,92,M,East Jerry,Wallis and Futuna,2225629684025639,"95949 Jill Circle, Tuckerchester, VI 49522",2023-05-31,2023-06-26,...,2023-06-30,Yearly,FictionTales,Books,Fiction,994,2021-05-08,-1,5,Almost account important charge oil style anyone.


In [12]:
# Shuffle the data
df_shuffled = df.sample(frac=1, random_state=42)


#input db_percentage et json_percentage
json_percentage = float(input('Enter json percentage from 0 to 1: '))
db_percentage = float(input('Enter database percentage from 0 to 1: '))

# Calculate the number of rows for each destination
total_rows = df_shuffled.shape[0]
json_rows = int(json_percentage * total_rows)
database_rows = int(db_percentage * total_rows)
csv_rows = total_rows - json_rows - database_rows

# Split the data into JSON, database, and CSV parts
df_json = df_shuffled.iloc[:json_rows]
df_database = df_shuffled.iloc[json_rows:json_rows+database_rows]
df_csv = df_shuffled.iloc[json_rows+database_rows:]

Enter json percentage from 0 to 1: 0.3
Enter database percentage from 0 to 1: 0.4


In [13]:
df_database.head()

,User ID,Subscription Type,Monthly Revenue,Country,Age,Gender,Device,Plan Duration
1552,1553,Standard,12,United States,39,Male,Tablet,1 Month
2290,2291,Premium,11,United States,31,Female,Laptop,1 Month
1398,1399,Premium,12,Spain,31,Female,Smart TV,1 Month
1775,1776,Premium,15,Spain,27,Female,Smart TV,1 Month
2299,2300,Basic,13,Italy,42,Female,Tablet,1 Month


In [14]:
# Save the splits to their respective destinations (JSON, database, and CSV)
df_json.to_json('ecommerce-data-json.json', orient='records', lines=True)
df_csv.to_csv('ecommerce-data-csv.csv', index=False, header=False)

In [15]:
# Créez la connexion
connection = pyodbc.connect('DRIVER={SQL SERVER} ;SERVER=LAPTOP-QJ16KG8Q\SQLEXPRESS;DATABASE=Netflixuserbase')

# Create a cursor to execute SQL queries
cursor = connection.cursor()

In [16]:
cursor.execute('''
    CREATE TABLE [ecommerce-data] (
        [UserID] INT,
        [Subscription Type] NVARCHAR(55),
        [Monthly Revenue] INT,
        [Join Date] DATE,
        [Last Payment] DATE,
        [Country] NVARCHAR(55),
        [Age] INT,
        [Gender] NVARCHAR(55),
        [Device] NVARCHAR(55),
        [Plan Duration] NVARCHAR(55),
        PRIMARY KEY ([UserID])
    );
''')

In [17]:
# Insert data into the dbpart table
for row in df_database.itertuples():
    # Check if the person_id already exists in the table
    cursor.execute('SELECT COUNT(*) FROM [ecommerce-data] WHERE [UserID] = ?', (row[1],))
    count = cursor.fetchone()[0]
    
    if count == 0:
        # Insert the record if it doesn't exist
        cursor.execute('INSERT INTO ecommerce-data ([UserID], [Subscription Type], [Monthly Revenue], [Country], [Age], [Gender], [Device], [Plan Duration]) VALUES (?, ?, ?, ?, ?, ?, ?, ?)', (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))

connection.commit()

In [19]:
# Close the connection
cursor.close()
connection.close()

ProgrammingError: Attempt to use a closed cursor.